Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Комментарий ревьюера
<span style="color:green">Начнём.</span>

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from random import randint
import seaborn as sb
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Первым делом необходимо считать датасет

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

Смотрю информацию по датасету, из которой можно заметить, что пропусков в данных нет

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Далее я переименовываю названия столбцов для более удобного обращения

In [4]:
df.rename(columns={'Пол': 'gender', 
                   'Возраст': 'age', 
                   'Зарплата': 'salary', 
                   'Члены семьи': 'relatives', 
                   'Страховые выплаты': 'insurance_payments'}, inplace=True)

Смотрю количество повторяющихся значений

In [5]:
df.duplicated().sum()

153

Удаляю повторяющиеся значения

In [6]:
df.drop_duplicates(inplace=True)

Разбиваю датасет на признаки и результат

In [7]:
X = df.drop('insurance_payments', axis=1)
y = df['insurance_payments'] 

Накидал функцию по апсемплингу, так как есть проблема (insurance_payments = 5 встречается всего один раз). Пока ее не использовал, так как не уверен что это необходимо, и вызывает сомнение корректность ее работы

In [8]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    features_two = features[target == 2]
    features_three = features[target == 3]
    features_four = features[target == 4]
    features_five = features[target == 5]
    
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    target_two = target[target == 2]
    target_three = target[target == 3]
    target_four = target[target == 4]
    target_five = target[target == 5]

    features_upsampled = pd.concat([features_zeros] + [features_ones] + [features_two] + [features_three] + [features_four] + [features_five] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] + [target_two] + [target_three] + [target_four] + [target_five] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [9]:
#X, y = upsample(X, y, 10)

Разбиваю датасет на обучение и тест. Дисбаланс классов не учитываю

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)#, stratify=y)

### Выводы
- Считан датасет
- Произведена предобработка, а именно: переименованы столбцы, найдены и удалены дублирующиеся строки
- Датасет разбит на обучающую и тестовую выборку

## 2. Умножение матриц

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Для ответа на поставленный вопрос было решено поступить максимально просто - обучить модель на исходных даных, оценить ее, после этого обучить другую модель на закодированных данных, оценить ее и сравнить полченные результаты

Для начала в формуле обучения домножим матрицу X на сгенерированную матрицу M

$$
w = ((X M)^T X M)^{-1} (X M)^T y
$$

Согласно свойству транспонирования ($ (X Y)^T = Y^T X^T $) раскроем скобки с транспонированием 

$$
w = (M^T X^T X M)^{-1} M^T X^T y
$$

Согласно свойству обратных матриц получаем следующее выражение

$$
w  = M^{-1} (X^T X)^{-1} (M^T)^{-1} M^T X^T y
$$

Подставим полученное выражение в формулу предсказания (не забываем, что X поменялся на X M):

$$
a = X M M^{-1} (X^T X)^{-1} (M^T)^{-1} M^T X^T y
$$

Согласно свойству $ A A^{-1} = A^{-1} A = E упростим выражение:

$$
a = X E (X^T X)^{-1} E X^T y
$$

Так как при умножении на еденичну матрицу исходная не изменится, то единичную матрицу можно откинуть:

$$
a = X (X^T X)^{-1} X^T y
$$

Что, согласно представленным формулам превращается в $ a = X w $ 

In [11]:
model = LinearRegression()

In [12]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
predict = model.predict(X_test)

In [14]:
mean_squared_error(y_test, predict)

0.12311255314500207

Как можно заметить, MSE в данном случае = 0.12311255314500207. Качество модели нас не интресует, инересует лишь значение MSE

Нахожу размер матрицы для получения матрицы - "ключа шифрования"

In [15]:
X_train.shape

(3877, 4)

Генерирую матрицу, при помощи которой матрица признаков буде закодирована

In [16]:
a = np.random.randint(10, size=(4,4))

Проверяю ее на обратимость

In [17]:
np.linalg.inv(a)

array([[ 0.11616848,  0.05978261, -0.05706522,  0.02105978],
       [ 0.13586957, -0.30434783,  0.10869565, -0.01630435],
       [-0.04279891,  0.13586957, -0.08423913,  0.15013587],
       [-0.16847826,  0.2173913 ,  0.06521739, -0.05978261]])

Перемножаю обучающюю и тестовую матрицу на сгенерированную матрицу

In [18]:
tmp_train = X_train @ a

In [19]:
tmp_test = X_test @ a

Заново обучаю модель на закодированных входных параметрах и нахожу MSE

In [20]:
model_test = LinearRegression()
model_test.fit(tmp_train, y_train)
predict_test = model_test.predict(tmp_test)
mean_squared_error(y_test, predict_test)

0.12311255314500207

In [21]:
predict

array([-1.11878714e-01,  2.19639096e-01,  2.22093806e-01,  4.22988461e-01,
        6.79695607e-01, -5.53474783e-02,  4.04141078e-01,  7.11492653e-01,
        7.47217874e-01,  3.16151517e-02,  1.14601735e-01, -1.86339566e-02,
        2.51429085e-01,  9.75621742e-02,  5.80844511e-01,  5.50420154e-01,
        2.26976031e-02,  6.76458478e-02, -1.88072998e-01,  4.20131394e-01,
       -1.68909543e-02,  4.06505962e-01,  4.37925637e-01,  6.97482320e-02,
        4.05637420e-02,  7.93584676e-01,  5.49593575e-01, -1.56701175e-01,
        3.31756242e-01, -1.56934773e-01,  2.08827440e-02,  1.29154523e-01,
        2.45936597e-01, -6.88221211e-02,  9.94309602e-02,  1.00609884e-01,
        5.62661965e-01,  5.19000623e-02,  5.26767985e-01,  2.14428058e-01,
       -2.98519038e-01, -1.17784491e-01, -2.14993382e-01,  2.09420736e-01,
        1.30843618e-01, -2.29211795e-01, -2.59595317e-01, -1.85359665e-01,
        1.50470709e-01,  3.39080577e-01,  2.08098059e-01, -2.69950374e-01,
        3.67840845e-01, -

Как можно заметить, MSE на исходных признаках равна MSE на закодированных признаках

**Ответ:** При умножении матрицы входных признаков на любую обратимую матрицу качество предсказания **не изменится**

**Обоснование:** В данном блоке было обучено две модели SGDRegressor (для примера). Качество модели оценивалось при помощи MSE. Как можно заметить, MSE на исходной матрице признаков и на преобразованной матрице признаков равна между собой. 

## 3. Алгоритм преобразования

**Алгоритм**

Алгоритм преобразования входного признакового пространства выбран как и в предыдущем пункте, а именно умножения исходного входного признакового пространства на любую обратимую матрицу. 
Шаги алгоримта:
1. Найти размеры матрицы входных признаков
2. Сгенерировать квадратную обратимую матрицу, размерами равными количеству признаков
3. Найти скалярное произведение матрицы признаков и сгенерированной матрицы
4. Полученое скалярное произведение матриц будет являтся новой матрицей входных признаков 

**Обоснование**

Ниже перечислены причины, почему был выбран этот алгоритм:
- Его эффективность доказана в предыдущем пункте
- На данный момент я пока что не так силен в линейной алгебре и статистике. Была идея сравнить две матрицы (исходную и преобразованную) на их "похожесть" каким-то стаистическим инструментом, но пока что я не нашел подходящий, буду надеяся на возможную подсказку куда копать:)
- Ограниченное время на исполнение проекта

## 4. Проверка алгоритма

Далее делаются те же шаги, что и в пункте 2: обучение, предсказание, нахождение R2 сначала для исходных признаков, затем для закодированных

In [22]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2_score(y_test, y_pred)

0.41605492161510915

In [23]:
model = LinearRegression()
model.fit(tmp_train, y_train)
y_pred = model.predict(tmp_test)
r2_score(y_test, y_pred)

0.41605492161503554

R2 на исходных признаках и на закодированных сходятся в первых 12 символах

# Общие выводы
- Датасет предобработан и разбит на обучающую и тестовую выборки
- Найден ответ на поставленный вопрос
- Выбран алгоритм кодирования
- метрика R2 для модели линейной регрессии на исходных параметрах оказалась равной в первых 12 знаках метрике R2 на закодированнных признаках